In [216]:
import numpy as np
#from numpy.linalg import norm
import os
from scipy.linalg import eigh, qr, null_space, norm
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy.sparse import eye, kron, identity, csr_matrix, csc_matrix, lil_matrix, dok_matrix, issparse, coo_matrix
from scipy.sparse.linalg import eigsh, eigs, lobpcg, LinearOperator, ArpackNoConvergence
from scipy.optimize import curve_fit
from qutip import Qobj, ptrace, entropy_vn, qeye, tensor
from tqdm import tqdm
from itertools import product
from functools import reduce
import torch
import torch.optim as optim
from torch.autograd import Variable
import sympy as sp
from collections import Counter
from IPython.display import display, HTML

In [210]:
def pauli_x():
    """Pauli X matrix."""
    return np.array([[0, 1], [1, 0]])

def pauli_z():
    """Pauli Z matrix."""
    return np.array([[1, 0], [0, -1]])

def icosahedral_bonds(): #12 vertices
    """
    Defines the connectivity of a true 12-vertex icosahedral molecular structure.
    
    Returns:
        list of tuples: Each tuple (i, j) represents a bond between spin i and spin j.
    """
    bonds = [
        (0, 2), (0, 4), (0, 5), (0, 8), (0, 9),
        (1, 3), (1, 6), (1, 7), (1, 10), (1, 11),
        (2, 6), (2, 7), (2, 8), (2, 9), (3, 4),
        (3, 5), (3, 10), (3, 11), (4, 5), (4, 8),
        (4, 10), (5, 9), (5, 11), (6, 7), (6, 8),
        (6, 10), (7, 9), (7, 11), (8, 10), (9, 11)
    ]
    return bonds


def transverse_field_ising_icosahedral(N, J, h):
    """
    Constructs the Hamiltonian for the transverse field Ising model on an icosahedral molecular structure.
    
    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        h (float): Transverse field strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 12:
        raise ValueError("Icosahedral molecules typically have N = 12 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    X = csr_matrix(pauli_x())
    Z = csr_matrix(pauli_z())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)
    
    # Get icosahedral bonds
    bonds = icosahedral_bonds()
    
    # Interaction term: J * sigma_i^x * sigma_j^x for icosahedral connectivity
    for i, j in bonds:
        term = 1
        for k in range(N):
            if k == i or k == j:
                term = kron(term, X, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += J * term
    
    # Transverse field term: -h * sigma_i^z
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = kron(term, Z, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += -h * term
    
    return H

def ising_icosahedron(N, J):
    """
    Constructs the Hamiltonian for the transverse field Ising model on an icosahedral molecular structure without transverse field.
    
    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        """
    if N != 12:
        raise ValueError("Icosahedral molecules typically have N = 12 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    X = csr_matrix(pauli_x())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)
    
    # Get icosahedral bonds
    bonds = icosahedral_bonds()
    
    # Interaction term: J * sigma_i^x * sigma_j^x for icosahedral connectivity
    for i, j in bonds:
        term = 1
        for k in range(N):
            if k == i or k == j:
                term = kron(term, X, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += J * term
    
    return H

def transverse_field_icosahedral(N, h):
    """
    Constructs the Hamiltonian for the transverse field Ising model on an icosahedral molecular structure.
    
    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        h (float): Transverse field strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 12:
        raise ValueError("Icosahedral molecules typically have N = 12 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    Z = csr_matrix(pauli_z())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)
    
    # Get icosahedral bonds
    bonds = icosahedral_bonds()
    
    # Transverse field term: -h * sigma_i^z
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = kron(term, Z, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += -h * term
    
    return H

#######################################################################################################################

'''
def partial_trace_qubit(rho, keep, dims):
    """Compute the partial trace of a density matrix of qubits."""
    keep_dims = np.prod([dims[i] for i in keep])
    trace_dims = np.prod([dims[i] for i in range(len(dims)) if i not in keep])
    rho = rho.reshape([keep_dims, trace_dims, keep_dims, trace_dims])
    return np.trace(rho, axis1=1, axis2=3).reshape([keep_dims, keep_dims])

def partial_trace_qubit_torch(rho, keep, dims):
    """Compute the partial trace of a density matrix of qubits using PyTorch."""
    keep_dims = torch.prod(torch.tensor([dims[i] for i in keep]))
    trace_dims = torch.prod(torch.tensor([dims[i] for i in range(len(dims)) if i not in keep]))
    rho = rho.view(keep_dims, trace_dims, keep_dims, trace_dims)
    # Compute the partial trace
    traced_rho = torch.zeros((keep_dims, keep_dims), dtype=rho.dtype)
    for i in range(trace_dims):
        traced_rho += rho[:, i, :, i]
    #return traced_rho.view(keep_dims, keep_dims)
    return traced_rho'''

def isket_numpy(arr):
    """
    Check if a NumPy array is a ket (column vector).

    Parameters:
    - arr: np.ndarray, the array to check.

    Returns:
    - bool, True if the array is a ket, False otherwise.
    """
    if not isinstance(arr, np.ndarray):
        raise ValueError("Input must be a NumPy array")

    shape = arr.shape

    if len(shape) == 2 and shape[1] == 1:
        return True
    else:
        return False

def ptrace_numpy(Q, sel, dims): # numpy function adapted from ptrace of qutip
    """
    Compute the partial trace of a density matrix of qubits using NumPy.

    Parameters:
    - Q: numpy object, the quantum object (density matrix or state vector).
    - sel: list of int, indices of the subsystems to keep.
    - dims: list of int, dimensions of the subsystems.

    Returns:
    - numpy object, the reduced density matrix after tracing out the specified subsystems.
    """
    # Get the dimensions of the subsystems
    rd = np.asarray(dims[0], dtype=np.int32).ravel()
    nd = len(rd)
    
    # Ensure sel is a sorted array of indices
    if isinstance(sel, int):
        sel = np.array([sel])
    else:
        sel = np.asarray(sel)
    sel = list(np.sort(sel))
    
    # Dimensions of the subsystems to keep
    dkeep = (rd[sel]).tolist()
    
    # Indices of the subsystems to trace out
    qtrace = list(set(np.arange(nd)) - set(sel))
    
    # Dimensions of the subsystems to trace out
    dtrace = (rd[qtrace]).tolist()
    
    # Reshape the density matrix or state vector
    rd = list(rd)
    if isket_numpy(Q):
        # Reshape and transpose for state vector
        vmat = (Q
                .reshape(rd)
                .transpose(sel + qtrace)
                .reshape([np.prod(dkeep), np.prod(dtrace)]))
        # Compute the reduced density matrix
        rhomat = vmat.dot(vmat.conj().T)
    else:
        # Reshape and transpose for density matrix
        rhomat = np.trace(Q
                          .reshape(rd + rd)
                          .transpose(qtrace + [nd + q for q in qtrace] +
                                     sel + [nd + q for q in sel])
                          .reshape([np.prod(dtrace),
                                    np.prod(dtrace),
                                    np.prod(dkeep),
                                    np.prod(dkeep)]))
    return rhomat


def ptrace_sparse(psi_sparse, keep, dims):
    """
    Compute the partial trace over arbitrary subsystems using sparse matrix operations.

    Args:
        psi_sparse (scipy.sparse matrix): Full density matrix of shape (D, D), where D = product(dims)
        keep (list of int): Subsystems to keep (indices, 0-indexed)
        dims (list of int): List of subsystem dimensions, e.g., [2]*n for n qubits

    Returns:
        scipy.sparse.csr_matrix: Reduced density matrix over kept subsystems
    """
    if not issparse(psi_sparse):
        raise ValueError("psi_sparse must be a scipy.sparse matrix")
    n = len(dims)
    D = np.prod(dims)
    if psi_sparse.shape != (D, D):
        raise ValueError("Density matrix shape does not match dims")
    trace = [i for i in range(n) if i not in keep]
    d_keep = np.prod([dims[i] for i in keep])
    # Prepare output
    data = []
    row_idx = []
    col_idx = []

    # Precompute bit masks
    def idx_to_bits(idx):
        return np.array(list(np.binary_repr(idx, width=n))).astype(int)
    

    psi_sparse = psi_sparse.tocoo()
    for i, j, val in zip(psi_sparse.row, psi_sparse.col, psi_sparse.data):
        bi = idx_to_bits(i)
        bj = idx_to_bits(j)


        # Only sum terms where traced-out subsystems agree
        if np.all(bi[trace] == bj[trace]):
            # Extract kept bits and convert to reduced indices
            #print('condition met for i, j:', i, j)
            i_red_bits = bi[keep]
            j_red_bits = bj[keep]
            i_red = int("".join(i_red_bits.astype(str)), 2)
            j_red = int("".join(j_red_bits.astype(str)), 2)


            data.append(val)
            row_idx.append(i_red)
            col_idx.append(j_red)
    
    return coo_matrix((data, (row_idx, col_idx)), shape=(d_keep, d_keep)).tocsr()


def isket_torch(arr):
    """
    Check if a PyTorch tensor is a ket (column vector).

    Parameters:
    - arr: torch.Tensor, the array to check.

    Returns:
    - bool, True if the array is a ket, False otherwise.
    """
    if not isinstance(arr, torch.Tensor):
        raise ValueError("Input must be a PyTorch tensor")

    shape = arr.shape

    if len(shape) == 2 and shape[1] == 1:
        return True
    else:
        return False

def ptrace_torch(Q, sel, dims): # torch function adapted from ptrace of qutip
    """
    Compute the partial trace of a density matrix of qubits using PyTorch.

    Parameters:
    - Q: torch.Tensor, the quantum object (density matrix or state vector).
    - sel: list of int, indices of the subsystems to keep.
    - dims: list of int, dimensions of the subsystems.

    Returns:
    - torch.Tensor, the reduced density matrix after tracing out the specified subsystems.
    """
    # Get the dimensions of the subsystems
    rd = torch.tensor(dims[0], dtype=torch.int32).flatten()
    nd = len(rd)
    #print("rd", rd)
    #print("nd", nd)
    
    # Ensure sel is a sorted array of indices
    if isinstance(sel, int):
        sel = torch.tensor([sel])
    else:
        sel = torch.tensor(sel)
    sel = torch.sort(sel).values.tolist()
    
    # Dimensions of the subsystems to keep
    dkeep = rd[sel].tolist()
    
    # Indices of the subsystems to trace out
    qtrace = list(set(range(nd)) - set(sel))
    
    # Dimensions of the subsystems to trace out
    dtrace = rd[qtrace].tolist()
    
    # Reshape the density matrix or state vector
    rd = rd.tolist()
    if isket_torch(Q):
        # Reshape and transpose for state vector
        reshaped_Q = Q.reshape(rd)
        #print(reshaped_Q.shape)
        transposed_Q = reshaped_Q.permute(sel + qtrace)
        #print(transposed_Q.shape)
        vmat = transposed_Q.reshape([torch.prod(torch.tensor(dkeep)), torch.prod(torch.tensor(dtrace))])
        #print(vmat.shape)
        # Compute the reduced density matrix
        rhomat = vmat @ vmat.conj().T
        #print(rhomat.shape)
    else:
        # Reshape and transpose for density matrix
        reshaped_Q = Q.reshape(rd + rd)
        #print("reshaped_Q", reshaped_Q.shape)
        transposed_Q = reshaped_Q.permute(qtrace + [nd + q for q in qtrace] + sel + [nd + q for q in sel])
        #print("transposed_Q", transposed_Q.shape)
        reshaped_transposed_Q = transposed_Q.reshape([torch.prod(torch.tensor(dtrace)), torch.prod(torch.tensor(dtrace)), torch.prod(torch.tensor(dkeep)), torch.prod(torch.tensor(dkeep))])
        #print("reshaped_transposed_Q", reshaped_transposed_Q.shape)
        #rhomat = torch.trace(reshaped_transposed_Q)
        rhomat = torch.einsum('iikl->kl', reshaped_transposed_Q)
        # Trace out the first two dimensions
        #rhomat = torch.zeros((torch.prod(torch.tensor(dkeep)), torch.prod(torch.tensor(dkeep))), dtype=Q.dtype)
        #for i in range(reshaped_transposed_Q.shape[0]):
        #    for j in range(reshaped_transposed_Q.shape[1]):
        #        rhomat += reshaped_transposed_Q[i, j, :, :]
        #print("rhomat", rhomat.shape)
    return rhomat

def entanglement_entropy(psi, subsystem, total_size):

    '''Computes the bipartite entanglement entropy of a pure state.
    
    Parameters:
    psi : np.array
        The wavefunction (state vector) of the full system.
    subsystem_size : int
        The number of qubits in subsystem A.
    total_size : int
        The total number of qubits in the system.
    
    Returns:
    float
        The von Neumann entanglement entropy S_A.'''
    
    psi_matrix =  np.outer(psi, psi.conj())

    # Compute the reduced density matrix rho_A = Tr_B(|psi><psi|)
    rho_A = ptrace_numpy(psi_matrix, subsystem, [[2]*total_size, [2]*total_size])  # Partial trace over B
    
    # Compute eigenvalues of rho_A
    eigenvalues = np.linalg.eigvalsh(rho_A)
    
    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigenvalues = eigenvalues[eigenvalues > 0]
    
    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -np.sum(eigenvalues * np.log2(eigenvalues))
    
    return entropy

def entanglement_entropy_torch(psi, subsystem, total_size):
    """
    Computes the bipartite entanglement entropy of a pure state using PyTorch.

    Parameters:
    - psi: torch.Tensor (complex), the wavefunction (state vector) of the full system.
    - subsystem_size: int, the number of qubits in subsystem A.
    - total_size: int, the total number of qubits in the system.

    Returns:
    - torch.Tensor (scalar), the von Neumann entanglement entropy S_A.
    """

    if not isinstance(psi, torch.Tensor):
        psi = torch.tensor(psi, dtype=torch.complex64)
    
    # Ensure psi is normalized
    psi = psi / torch.norm(psi)

    # Compute the density matrix |psi><psi|
    psi_matrix = torch.outer(psi, psi.conj())

    # Compute the reduced density matrix rho_A = Tr_B(|psi><psi|)
    rho_A = ptrace_torch(psi_matrix, subsystem, [[2] * total_size, [2] * total_size])  # Partial trace over B

    #rho_A = rho_A.to(dtype=torch.float64)
    
    # Compute eigenvalues of rho_A
    eigvals = torch.linalg.eigvalsh(rho_A)

    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigvals = eigvals[eigvals > 0]

    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -torch.sum(eigvals * torch.log2(eigvals))

    return entropy

def entanglement_entropy_qutip(psi, subsystem, total_size):
    
    # Convert the wavefunction to a QuTiP Qobj
    density_matrix = np.outer(psi, psi.conj())
    density_matrix_qobj = Qobj(density_matrix, dims=[[2]*total_size, [2]*total_size])

    rho_A = ptrace(density_matrix_qobj, subsystem)
    # Compute the von Neumann entropy S_A
    entropy = entropy_vn(rho_A, base=2)
    
    return entropy

def entanglement_entropy_np_ptrace(rdm):
    # rdm already computed and converted to numpy
    # Compute eigenvalues of rho_A
    eigenvalues = np.linalg.eigvalsh(rdm)
    
    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigenvalues = eigenvalues[eigenvalues > 0]
    
    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -np.sum(eigenvalues * np.log2(eigenvalues))
    
    return entropy

def entanglement_entropy_torch_ptrace(rdm):

    eigvals = torch.linalg.eigvalsh(rdm)
    eigvals = eigvals[eigvals > 0]
    entropy = -torch.sum(eigvals * torch.log2(eigvals))
    return entropy


def entanglement_entropy_qutip_torch(psi, N):
    """
    Compute the von Neumann entanglement entropy using qutip.

    Parameters:
    - psi: torch.Tensor (complex), state vector of a quantum system.
    - N: int, total number of qubits.

    Returns:
    - torch.Tensor (scalar), von Neumann entropy.
    """
    # Ensure psi is normalized
    psi = psi / torch.norm(psi)

    # Convert PyTorch tensor to NumPy for QuTiP
    psi_np = psi.detach().numpy()

    rho_np = np.outer(psi_np, psi_np.conj())
    rho_qobj = Qobj(rho_np, dims=[[2] * N, [2] * N])

    rho_A = ptrace(rho_qobj, list(range(N // 2)))

    # Compute von Neumann entropy
    entropy = entropy_vn(rho_A, base=2)  # Compute in log base 2

    # Convert back to PyTorch tensor to allow gradient flow
    return torch.tensor(entropy, dtype=torch.float32, requires_grad=True)

#######################################################################################################################

# Define the linear combination function - numpy
def linear_combination_np(coeffs, psis):
    # Ensure psis are numpy tensors
    psi_np = [np.array(psi) for psi in psis]
    # Compute the linear combination in PyTorch
    psi = sum(c * psi for c, psi in zip(coeffs, psis))
    
    return psi

# Define the linear combination function - torch
def linear_combination(coeffs, psis):
    # Ensure psis are PyTorch tensors
    psis_torch = [torch.tensor(psi, dtype=torch.complex64) if not isinstance(psi, torch.Tensor) else psi for psi in psis]
    
    # Compute the linear combination in PyTorch
    psi_torch = sum(c * psi for c, psi in zip(coeffs, psis_torch))
    
    return psi_torch

# Define the linear combination function - torch but after computing the ptrace of outer products of scars
def linear_combination_outer(coeffs, outs):
    # Ensure outs are PyTorch tensors
    outs_torch = [torch.tensor(out, dtype=torch.complex64) if not isinstance(out, torch.Tensor) else out for out in outs]
    torch_coeffs = torch.tensor(coeffs, dtype=torch.complex64)

    # Compute the PyTorch tensor of out_coeffs which is the product of all possible combinations of c_i^* times c_j
    out_coeffs = torch.zeros((len(torch_coeffs), len(torch_coeffs)), dtype=torch.complex64)
    for i in range(len(torch_coeffs)):
        for j in range(len(torch_coeffs)):
            out_coeffs[i, j] = torch.conj(torch_coeffs[i]) * torch_coeffs[j]
    
    # Compute the linear combination in PyTorch
    lin_torch = sum(out_coeffs[i, j] * outs_torch[i] for i in range(len(coeffs)) for j in range(len(coeffs)))
    
    return lin_torch

######################################################

# chebyshev

def jackson_weights(m):
    """
    Jackson damping coefficients for k = 0..m.
    (You can replace this with your own implementation if you already have one.)
    """
    k = np.arange(m+1, dtype=float)
    N = m + 1.0
    # Standard Jackson kernel for Chebyshev series
    # g_k = [(N - k + 1) * cos(pi*k/(N+1)) + sin(pi*k/(N+1)) / tan(pi/(N+1))] / (N+1)
    gk = ((N - k + 1) * np.cos(np.pi * k / (N + 1.0)) +
          np.sin(np.pi * k / (N + 1.0)) / np.tan(np.pi / (N + 1.0))) / (N + 1.0)
    return gk

def chebyshev_filter_numpy(H, Emin, Emax, target_E0, m,
                           pad=0.05, use_jackson=True, rng=None):
    """
    Chebyshev cosine kernel filter, pure NumPy/SciPy version.

    Parameters
    ----------
    H : (n, n) array_like or sparse_matrix
        Real symmetric / Hermitian matrix.
    Emin, Emax : float
        Estimated spectral bounds of H.
    target_E0 : float
        Target energy where we want to focus the filter.
    m : int
        Polynomial degree.
    pad : float, optional
        Padding fraction for bounds.
    use_jackson : bool, optional
        Apply Jackson damping.
    rng : np.random.Generator, optional
        Random generator.

    Returns
    -------
    filt : ndarray, shape (n,)
        Normalized filtered vector.
    approx_E : float
        Rayleigh quotient <filt|H|filt>.
    """
    if rng is None:
        rng = np.random.default_rng()

    # 1) Padded bounds and rescaling parameters
    width  = Emax - Emin
    Emin_p = Emin - pad * width
    Emax_p = Emax + pad * width

    c = 0.5 * (Emax_p + Emin_p)
    d = 0.5 * (Emax_p - Emin_p)

    # 2) Rescaled target x0 and Chebyshev coefficients alpha_k
    x0 = (target_E0 - c) / d
    x0 = float(np.clip(x0, -0.999999, 0.999999))
    theta0 = np.arccos(x0)

    alpha = np.cos(np.arange(m+1) * theta0)
    if use_jackson:
        g = jackson_weights(m)
        alpha = alpha * g

    # Helper: matvec with Htilde = (H - c I)/d
    def Htilde_dot(v):
        Hv = H @ v   # works for dense or sparse
        return (Hv - c * v) / d

    # 3) Random start vector
    n = H.shape[0]
    v0 = rng.standard_normal(n)
    v0 /= norm(v0)

    # 4) Chebyshev recursion
    t0 = v0
    t1 = Htilde_dot(v0)

    filt = alpha[0] * t0 + alpha[1] * t1

    tkm1 = t0
    tk   = t1

    for k in range(2, m+1):
        tkp1 = 2.0 * Htilde_dot(tk) - tkm1
        filt = filt + alpha[k] * tkp1
        tkm1, tk = tk, tkp1

    # 5) Normalize and Rayleigh quotient
    filt_norm = norm(filt)
    if filt_norm == 0:
        raise RuntimeError("Filtered vector became zero; try different parameters.")
    filt /= filt_norm

    Hv = H @ filt
    approx_E = np.vdot(filt, Hv).real / np.vdot(filt, filt).real

    return filt, approx_E

def chebyshev_filter_v0_numpy(H, v0, Emin, Emax, target_E0, m,
                           pad=0.05, use_jackson=True, rng=None):
    """
    Chebyshev cosine kernel filter, pure NumPy/SciPy version.

    Parameters
    ----------
    H : (n, n) array_like or sparse_matrix
        Real symmetric / Hermitian matrix.
    v0 : (n,) array_like
        Initial vector to start the Chebyshev recursion.
    Emin, Emax : float
        Estimated spectral bounds of H.
    target_E0 : float
        Target energy where we want to focus the filter.
    m : int
        Polynomial degree.
    pad : float, optional
        Padding fraction for bounds.
    use_jackson : bool, optional
        Apply Jackson damping.
    rng : np.random.Generator, optional
        Random generator.

    Returns
    -------
    filt : ndarray, shape (n,)
        Normalized filtered vector.
    approx_E : float
        Rayleigh quotient <filt|H|filt>.
    """
    if rng is None:
        rng = np.random.default_rng()

    # 1) Padded bounds and rescaling parameters
    width  = Emax - Emin
    Emin_p = Emin - pad * width
    Emax_p = Emax + pad * width

    c = 0.5 * (Emax_p + Emin_p)
    d = 0.5 * (Emax_p - Emin_p)

    # 2) Rescaled target x0 and Chebyshev coefficients alpha_k
    x0 = (target_E0 - c) / d
    x0 = float(np.clip(x0, -0.999999, 0.999999))
    theta0 = np.arccos(x0)

    alpha = np.cos(np.arange(m+1) * theta0)
    if use_jackson:
        g = jackson_weights(m)
        alpha = alpha * g

    # Helper: matvec with Htilde = (H - c I)/d
    def Htilde_dot(v):
        Hv = H @ v   # works for dense or sparse
        return (Hv - c * v) / d

    # 3) Normalize random start vector if not already normalized
    v0 /= norm(v0)

    # 4) Chebyshev recursion
    t0 = v0
    t1 = Htilde_dot(v0)

    filt = alpha[0] * t0 + alpha[1] * t1

    tkm1 = t0
    tk   = t1

    for k in range(2, m+1):
        tkp1 = 2.0 * Htilde_dot(tk) - tkm1
        filt = filt + alpha[k] * tkp1
        tkm1, tk = tk, tkp1

    # 5) Normalize and Rayleigh quotient
    filt_norm = norm(filt)
    if filt_norm == 0:
        raise RuntimeError("Filtered vector became zero; try different parameters.")
    filt /= filt_norm

    Hv = H @ filt
    approx_E = np.vdot(filt, Hv).real / np.vdot(filt, filt).real

    return filt, approx_E

def chebyshev_filter_block_numpy(H, V0, Emin, Emax, target_E0, m,
                                 pad=0.05, use_jackson=True):
    """
    Block Chebyshev cosine kernel filter (pure NumPy/SciPy version).

    Parameters
    ----------
    H : (n, n) array_like or sparse matrix
        Real symmetric / Hermitian matrix.
    V0 : (n, p) array_like
        Initial block of p vectors (columns) to start the Chebyshev recursion.
        Columns should be linearly independent; they need not be orthonormal.
    Emin, Emax : float
        Estimated spectral bounds of H.
    target_E0 : float
        Target energy where we want to focus the filter.
    m : int
        Polynomial degree.
    pad : float, optional
        Padding fraction for bounds.
    use_jackson : bool, optional
        Apply Jackson damping to the Chebyshev coefficients.

    Returns
    -------
    Phi : ndarray, shape (n, p)
        Approximate eigenvectors (columns) near target_E0.
    evals : ndarray, shape (p,)
        Corresponding Ritz eigenvalues.
    """

    V0 = np.array(V0, dtype=np.complex128, copy=True)
    n, p = V0.shape

    # 1) Padded bounds and rescaling parameters
    width  = Emax - Emin
    Emin_p = Emin - pad * width
    Emax_p = Emax + pad * width

    c = 0.5 * (Emax_p + Emin_p)
    d = 0.5 * (Emax_p - Emin_p)

    # 2) Rescaled target x0 and Chebyshev coefficients alpha_k
    x0 = (target_E0 - c) / d
    x0 = float(np.clip(x0, -0.999999, 0.999999))
    theta0 = np.arccos(x0)

    alpha = np.cos(np.arange(m+1) * theta0)
    if use_jackson:
        g = jackson_weights(m)   # assumed defined elsewhere
        alpha = alpha * g

    # Helper: Htilde = (H - c I)/d acting on a block
    def Htilde_dot_block(V):
        HV = H @ V              # works for dense or sparse
        return (HV - c * V) / d

    # 3) Orthonormalize starting block: V0 -> Q0
    #    (this gives us an orthonormal basis of the initial subspace)
    Q0, _ = np.linalg.qr(V0)    # (n, p), orthonormal columns

    # 4) Block Chebyshev recursion
    T0 = Q0                     # (n, p)
    T1 = Htilde_dot_block(Q0)   # (n, p)

    filt = alpha[0] * T0 + alpha[1] * T1

    Tkm1 = T0
    Tk   = T1

    for k in range(2, m+1):
        Tkp1 = 2.0 * Htilde_dot_block(Tk) - Tkm1
        filt = filt + alpha[k] * Tkp1
        Tkm1, Tk = Tk, Tkp1

    # 5) Orthonormalize the filtered block
    Q, _ = np.linalg.qr(filt)   # (n, p), orthonormal columns spanning filtered subspace

    # 6) Rayleigh–Ritz in the filtered subspace
    # H_sub is the projected matrix H in basis Q
    H_sub = Q.conj().T @ (H @ Q)    # (p, p)
    evals, U = np.linalg.eigh(H_sub)

    # 7) Lift Ritz eigenvectors back to full space
    Phi = Q @ U    # (n, p)

    return Phi, evals

### symmetry sectors -- Ih NEEDS TO BE ADDED

def check_magnetization_sector(vec, N, tol=1e-6): ### total magnetization is not conserved --- it only applies to scars
    """Check average magnetization of a state vector."""
    D = 1 << N
    mag_avg = 0.0
    for b in range(D):
        mag_avg += magnetization(b, N) * np.abs(vec[b])**2
    return mag_avg

def check_parity_sector(vec, N, tol=1e-6):
    """Check if vector is in even/odd parity sector."""
    D = 1 << N
    weight_even = sum(np.abs(vec[b])**2 for b in range(D) if parity(b, N) == 1)
    weight_odd = sum(np.abs(vec[b])**2 for b in range(D) if parity(b, N) == -1)
    if weight_even > 1.0 - tol:
        return "even"
    elif weight_odd > 1.0 - tol:
        return "odd"
    else:
        return f"mixed (even={weight_even:.4f}, odd={weight_odd:.4f})"

def build_parity_operator(N):
    """
    Build the parity operator as a matrix.
    Parity operator P|b> = (-1)^(number of 1s) |b>
    
    Parameters:
    - N: int, number of qubits
    
    Returns:
    - P_op: sparse matrix, parity operator (diagonal)
    """
    D = 1 << N
    diagonal = []
    
    for b in range(D):
        n_up = bin(b).count('1')
        # Even parity: +1, Odd parity: -1
        diagonal.append((-1)**n_up)
    
    return csr_matrix(np.diag(diagonal))

def commutator_norm(A, B):
    """
    Compute the Frobenius norm of the commutator [A, B] = AB - BA.
    For sparse matrices, use sparse operations.
    
    Parameters:
    - A, B: matrices (dense or sparse)
    
    Returns:
    - float, ||[A, B]||_F
    """
    comm = A @ B - B @ A
    
    if issparse(comm):
        # For sparse matrices, compute Frobenius norm
        return np.sqrt(comm.multiply(comm.conj()).sum())
    else:
        # For dense matrices
        return np.linalg.norm(comm, 'fro')

def magnetization(bitstring, N):
    # Suppose spin up = 1, spin down = 0
    # Or adjust convention as needed
    n_up = bitstring.bit_count()
    n_down = N - n_up
    return n_up - n_down  # proportional to total Sz

def parity(bitstring, N):
    """
    Compute parity of a bitstring.
    Returns +1 for even number of up spins, -1 for odd.
    """
    n_up = bitstring.bit_count()
    return 1 if (n_up % 2 == 0) else -1

def build_sz0_even_parity_indices(N):
    """
    Build indices for states with:
    - Zero magnetization (Sz = 0)
    - Even parity (even number of up spins)
    
    Parameters:
    - N: int, number of qubits
    
    Returns:
    - idx_sector: np.array, indices satisfying all conditions
    """
    D = 1 << N
    idx_sector = []
    
    for b in range(D):
        # Check magnetization = 0
        if magnetization(b, N) != 0:
            continue
        
        # Check even parity
        if parity(b, N) != 1:
            continue
        
        idx_sector.append(b)
    
    return np.array(idx_sector, dtype=np.int64)

def random_block_in_sz0_even_parity(N, block_size=5, rng=None):
    """
    Generate random block of vectors in the symmetry sector:
    - Sz = 0
    - Even parity
    
    Parameters:
    - N: int, number of qubits
    - block_size: int, number of vectors
    - rng: random generator
    
    Returns:
    - Q: (D, block_size) array, orthonormal columns in symmetry sector
    """
    if rng is None:
        rng = np.random.default_rng()

    D = 1 << N
    idx_sector = build_sz0_even_parity_indices(N)
    V = np.zeros((D, block_size), dtype=np.complex128)

    for k in range(block_size):
        v = np.zeros(D, dtype=np.complex128)
        
        # Random amplitudes in the sector
        amplitudes = rng.normal(size=len(idx_sector)) + 1j * rng.normal(size=len(idx_sector))
        
        # Assign random amplitudes to basis states in the sector
        v[idx_sector] = amplitudes
        
        V[:, k] = v

    # Orthonormalize columns
    Q, _ = np.linalg.qr(V)
    return Q   # shape (D, block_size)

In [242]:
N = 12  # Number of spins
J = 1.0  # Interaction strength
h = 3.0  # Transverse field strength # this is the value in the paper. maybe try  other values too, including the critical value one (h=J=1)

# Assuming transverse_field_ising is defined and returns a sparse Hermitian matrix
H = transverse_field_ising_icosahedral(N, J, h)
Hi = ising_icosahedron(N, J)
Htf = transverse_field_icosahedral(N, h)

print(f"Hamiltonian shape: {H.shape}")
print(f"Non-zero elements in H: {H.nnz}")

Hamiltonian shape: (4096, 4096)
Non-zero elements in H: 126052


In [212]:
# Build symmetry operators
P_op = build_parity_operator(N)

print("Checking if H commutes with symmetry operators:\n")

# Check [H, P] = 0 (parity symmetry)
comm_parity = commutator_norm(H, P_op)
print(f"Parity:")
print(f"  ||[H, P]||_F = {comm_parity:.6e}")
print(f"  ||H||_F = {H_norm:.6e}")
print(f"  Relative error: {comm_parity / H_norm:.6e}")
print(f"  Commutes: {'YES' if comm_parity / H_norm < 1e-10 else 'NO'}\n")

# Additional check: verify P^2 = Identity
P_squared = P_op @ P_op
identity = identity(1 << N, format='csr')

P_identity_error = np.sqrt((P_squared - identity).multiply((P_squared - identity).conj()).sum())

print(f"Operator properties:")
print(f"  ||P^2 - I||_F = {P_identity_error:.6e}")
print(f"  P is involutory: {'YES' if P_identity_error < 1e-10 else 'NO'}")

Checking if H commutes with symmetry operators:

Parity:
  ||[H, P]||_F = 0.000000e+00
  ||H||_F = 7.518298e+02
  Relative error: 0.000000e+00
  Commutes: YES

Operator properties:
  ||P^2 - I||_F = 0.000000e+00
  P is involutory: YES


In [213]:
bonds = [
    (0, 2), (0, 4), (0, 5), (0, 8), (0, 9),
    (1, 3), (1, 6), (1, 7), (1, 10), (1, 11),
    (2, 6), (2, 7), (2, 8), (2, 9),
    (3, 4), (3, 5), (3, 10), (3, 11),
    (4, 5), (4, 8), (4, 10),
    (5, 9), (5, 11),
    (6, 7), (6, 8), (6, 10),
    (7, 9), (7, 11),
    (8, 10),
    (9, 11),
]
bonds_set = {tuple(sorted(e)) for e in bonds}
print(len(bonds_set))
print(bonds_set)

# ============================================================
# 1. Load Ih permutations from file (old labelling)
# ============================================================

def load_ih_permutations(path="ih_permutations_0based.txt"):
    """
    Each line in ih_permutations_0based.txt is like:
    [0, 11, 10, 3, 6, 5, 4, 8, 7, 9, 2, 1]
    representing a permutation on vertices 0..11.
    """
    perms = []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            nums = line.strip('[]').split(',')
            perms.append(np.array([int(x) for x in nums], dtype=int))
    return perms

perms_old = load_ih_permutations("ih_permutations_0based.txt")
N_sites = len(perms_old[0])
print(f"Loaded {len(perms_old)} I_h permutations on {N_sites} sites (old labelling).")
print(perms_old[-1])

# ============================================================
# 2. Graph automorphism checker (for sanity)
# ============================================================

def is_automorphism(perm, bonds_set):
    """
    Check if 'perm' is a graph automorphism of the icosahedron
    defined by bonds_set, i.e. maps edges to edges.
    """
    for i, j in bonds_set:
        print("checking edge", (i, j))
        ii, jj = perm[i], perm[j]
        print("mapped to", (ii, jj))
        if tuple(sorted((ii, jj))) not in bonds_set:
            print("NOT AN AUTHOMORPHISM")
            return False
        else:
            print("AUTOMORPHISM")
    return True

print("Number of automorphisms in OLD labelling wrt your bonds:",
      sum(is_automorphism(p, bonds_set) for p in perms_old))


30
{(3, 4), (3, 10), (0, 2), (0, 5), (1, 6), (0, 8), (9, 11), (1, 3), (2, 8), (6, 8), (4, 5), (4, 8), (5, 9), (0, 4), (2, 7), (1, 11), (7, 9), (6, 7), (6, 10), (3, 5), (3, 11), (4, 10), (5, 11), (0, 9), (8, 10), (2, 9), (1, 7), (2, 6), (1, 10), (7, 11)}
Loaded 120 I_h permutations on 12 sites (old labelling).
[11  8  9 10  3  1  0  2  5  7  4  6]
checking edge (3, 4)
mapped to (3, 4)
AUTOMORPHISM
checking edge (3, 10)
mapped to (3, 10)
AUTOMORPHISM
checking edge (0, 2)
mapped to (0, 2)
AUTOMORPHISM
checking edge (0, 5)
mapped to (0, 5)
AUTOMORPHISM
checking edge (1, 6)
mapped to (1, 6)
AUTOMORPHISM
checking edge (0, 8)
mapped to (0, 8)
AUTOMORPHISM
checking edge (9, 11)
mapped to (9, 11)
AUTOMORPHISM
checking edge (1, 3)
mapped to (1, 3)
AUTOMORPHISM
checking edge (2, 8)
mapped to (2, 8)
AUTOMORPHISM
checking edge (6, 8)
mapped to (6, 8)
AUTOMORPHISM
checking edge (4, 5)
mapped to (4, 5)
AUTOMORPHISM
checking edge (4, 8)
mapped to (4, 8)
AUTOMORPHISM
checking edge (5, 9)
mapped to (5, 

In [254]:
bonds_set = {tuple(sorted(b)) for b in bonds}
print("Bonds set:", bonds_set)

for i, perm in enumerate(perms):  # each perm: length-N list/array of site labels
    bonds_transformed = {
        tuple(sorted((perm[a], perm[b]))) for (a, b) in bonds
    }
    print(f"Permutation {i} maps bonds to: {bonds_transformed}")

    if bonds_transformed == bonds_set:
        print("does preserve bonds")

Bonds set: {(3, 4), (3, 10), (0, 2), (0, 5), (1, 6), (0, 8), (9, 11), (1, 3), (2, 8), (6, 8), (4, 5), (4, 8), (5, 9), (0, 4), (2, 7), (1, 11), (7, 9), (6, 7), (6, 10), (3, 5), (3, 11), (4, 10), (5, 11), (0, 9), (8, 10), (2, 9), (1, 7), (2, 6), (1, 10), (7, 11)}
Permutation 0 maps bonds to: {(3, 4), (3, 10), (0, 2), (0, 5), (1, 6), (0, 8), (9, 11), (1, 3), (2, 8), (6, 8), (4, 5), (4, 8), (5, 9), (0, 4), (2, 7), (1, 11), (7, 9), (6, 7), (6, 10), (3, 5), (3, 11), (4, 10), (5, 11), (0, 9), (8, 10), (2, 9), (1, 7), (2, 6), (1, 10), (7, 11)}
does preserve bonds
Permutation 1 maps bonds to: {(3, 4), (3, 10), (0, 2), (0, 5), (1, 6), (0, 8), (9, 11), (1, 3), (2, 8), (6, 8), (4, 5), (4, 8), (5, 9), (0, 4), (2, 7), (1, 11), (7, 9), (6, 7), (6, 10), (3, 5), (3, 11), (4, 10), (5, 11), (0, 9), (8, 10), (2, 9), (1, 7), (2, 6), (1, 10), (7, 11)}
does preserve bonds
Permutation 2 maps bonds to: {(3, 4), (3, 10), (0, 2), (0, 5), (1, 6), (0, 8), (9, 11), (1, 3), (2, 8), (6, 8), (4, 5), (4, 8), (5, 9), (0

In [257]:
# ============================================================
# 0. Your icosahedron bonds (current labelling)
# ============================================================

bonds = [
    (0, 2), (0, 4), (0, 5), (0, 8), (0, 9),
    (1, 3), (1, 6), (1, 7), (1, 10), (1, 11),
    (2, 6), (2, 7), (2, 8), (2, 9),
    (3, 4), (3, 5), (3, 10), (3, 11),
    (4, 5), (4, 8), (4, 10),
    (5, 9), (5, 11),
    (6, 7), (6, 8), (6, 10),
    (7, 9), (7, 11),
    (8, 10),
    (9, 11),
]
bonds_set = {tuple(sorted(e)) for e in bonds}
print(bonds_set)

# ============================================================
# 1. Load Ih permutations from file
# ============================================================

def load_ih_permutations(path="ih_permutations_0based.txt"):
    """
    Each line in ih_permutations_0based.txt is like:
    [0, 11, 10, 3, 6, 5, 4, 8, 7, 9, 2, 1]
    representing a permutation on vertices 0..11.
    """
    perms = []
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            nums = line.strip('[]').split(',')
            perms.append(np.array([int(x) for x in nums], dtype=int))
    return perms

perms = load_ih_permutations("ih_permutations_0based.txt")
N_sites = len(perms[0])
print(f"Loaded {len(perms)} I_h permutations on {N_sites} sites.")

# ============================================================
# 2. Graph automorphism checker (for sanity)
# ============================================================

def is_automorphism(perm, bonds_set):
    """
    Check if 'perm' is a graph automorphism of the icosahedron
    defined by bonds_set, i.e. maps edges to edges.
    """
    for i, j in bonds_set:
        ii, jj = perm[i], perm[j]
        if tuple(sorted((ii, jj))) not in bonds_set:
            return False
    return True

print("Number of automorphisms wrt your bonds:",
      sum(is_automorphism(p, bonds_set) for p in perms))

# ============================================================
# 3. Permutation utilities & conjugacy classes of I_h
# ============================================================

def perm_compose(p, q):
    """
    Composition p ∘ q acting on indices [0..N-1]:
    result r satisfies r[i] = p[q[i]].
    """
    return p[q]

def perm_inverse(p):
    """Inverse permutation p^{-1}."""
    inv = np.empty_like(p)
    inv[p] = np.arange(len(p))
    return inv

def perm_order(p, max_iter=300):
    """Order of permutation p: smallest k>0 with p^k = identity."""
    n = len(p)
    e = np.arange(n)
    x = p.copy()
    k = 1
    while not np.array_equal(x, e):
        x = perm_compose(x, p)
        k += 1
        if k > max_iter:
            raise RuntimeError("Permutation order too large?")
    return k

def compute_conjugacy_classes(perms):
    """
    Compute conjugacy classes of the group represented by 'perms'
    via g -> h g h^{-1}.
    """
    perms_tuples = [tuple(p.tolist()) for p in perms]
    perm_dict = {pt: p for pt, p in zip(perms_tuples, perms)}

    unseen = set(perms_tuples)
    classes = []

    while unseen:
        rep_t = unseen.pop()
        rep = perm_dict[rep_t]
        current = set()

        for h_t, h in perm_dict.items():
            h_inv = perm_inverse(h)
            conj = perm_compose(h, perm_compose(rep, h_inv))
            conj_t = tuple(conj.tolist())
            if conj_t in perm_dict:
                current.add(conj_t)

        for ct in current:
            unseen.discard(ct)

        class_perms = [perm_dict[ct] for ct in current]
        classes.append(class_perms)

    return classes

classes = compute_conjugacy_classes(perms)
print(f"Found {len(classes)} conjugacy classes in new labelling.")
for i, cls in enumerate(classes):
    size = len(cls)
    order = perm_order(cls[0])
    print(f"class {i}: size={size:2d}, order={order:2d}")

# ============================================================
# 4. Build Hilbert-space operator U_g for a permutation g
# ============================================================

def build_symmetry_operator(N_spins, perm):
    """
    U |s_0 ... s_{N-1}> = |s_{perm(0)} ... s_{perm(N-1)}|
    with site 0 as the left-most tensor factor.
    """
    D = 1 << N_spins
    rows = np.empty(D, dtype=np.int64)
    cols = np.arange(D, dtype=np.int64)

    for b in range(D):
        # decode: big-endian, site 0 = most significant bit
        bits = [ (b >> (N_spins - 1 - i)) & 1 for i in range(N_spins) ]

        # permute sites
        permuted_bits = [ bits[perm[i]] for i in range(N_spins) ]

        # re-encode in the same big-endian convention
        b_prime = 0
        for i in range(N_spins):
            b_prime |= permuted_bits[i] << (N_spins - 1 - i)

        rows[b] = b_prime

    data = np.ones(D, dtype=np.int8)
    return csr_matrix((data, (rows, cols)), shape=(D, D))

def check_Ug(N_spins, Ug):

    dim = 1 << N_spins

    # 1. Size
    assert Ug.shape == (dim, dim), f"Ug has wrong shape: {Ug.shape}"

    # 2. Permutation structure: exactly one nonzero per row and per column
    # getnnz(axis=0/1) is # of nonzeros per column/row for CSR/CSC matrices
    nnz_per_row = Ug.getnnz(axis=1)
    nnz_per_col = Ug.getnnz(axis=0)

    #assert np.all(nnz_per_row == 1), "Some rows do not have exactly one '1'"
    #assert np.all(nnz_per_col == 1), "Some columns do not have exactly one '1'"

    # 3. Unitarity: Ug† Ug = I
    Id = identity(dim, dtype=np.complex128, format='csr')
    diff = (Ug.conj().T @ Ug) - Id
    # because entries are exact 0/1, this should be exactly zero
    print("Number of nonzeros in Ug†Ug - I:", diff.nnz)
    if diff.nnz != 0:
        raise ValueError("Ug is not unitary: Ug†Ug - I has nonzero entries")

    print("All tests passed for this Ug.")


N_spins = N_sites  # 12 for icosahedron
Ugs = [build_symmetry_operator(N_spins, perm) for perm in perms]
# check unitarity for all Ugs
for ug in Ugs:
    check_Ug(N_spins, ug)


# ============================================================
# 5. Build class operators C_k = sum_{g in class_k} U_g
# ============================================================

# Build class operators
def build_class_operators(N_spins, classes):
    """
    Build class operators C_k = sum_{g in class_k} U_g.
    Returns sparse CSR matrices.
    """
    class_ops = []
    for class_perms in classes:
        U_sum = None
        for perm in class_perms:
            U_g = build_symmetry_operator(N_spins, perm)
            if U_sum is None:
                # Initialize as CSR with complex dtype
                U_sum = U_g.astype(np.complex128, copy=True).tocsr()
            else:
                # Add sparse matrices (stays sparse)
                U_sum = U_sum + U_g.tocsr()
        # Ensure final result is CSR format
        class_ops.append(U_sum.tocsr())
    return class_ops


# ============================================================
# 6. Commutator norm and symmetry check
# ============================================================

Ugs = [build_symmetry_operator(N_spins, perm) for perm in perms]

def comm_norm(A, B):
# sparse Frobenius norm of commutator
    C = A @ B - B @ A
    return np.sqrt((C.multiply(C.conj())).sum())

for i, Ug in enumerate(Ugs):
    nrm = comm_norm(H, Ug)
    print(f"||[H, U_{i}]||_F = {nrm:.3e}")
    nrmi = comm_norm(Hi, Ug)
    print(f"||[Hi, U_{i}]||_F = {nrmi:.3e}")
    nrmtf = comm_norm(Htf, Ug)
    print(f"||[Htf, U_{i}]||_F = {nrmtf:.3e}")

{(3, 4), (3, 10), (0, 2), (0, 5), (1, 6), (0, 8), (9, 11), (1, 3), (2, 8), (6, 8), (4, 5), (4, 8), (5, 9), (0, 4), (2, 7), (1, 11), (7, 9), (6, 7), (6, 10), (3, 5), (3, 11), (4, 10), (5, 11), (0, 9), (8, 10), (2, 9), (1, 7), (2, 6), (1, 10), (7, 11)}
Loaded 120 I_h permutations on 12 sites.
Number of automorphisms wrt your bonds: 120
Found 10 conjugacy classes in new labelling.
class 0: size=12, order=10
class 1: size=12, order= 5
class 2: size=15, order= 2
class 3: size=20, order= 3
class 4: size=12, order=10
class 5: size=12, order= 5
class 6: size=20, order= 6
class 7: size=15, order= 2
class 8: size= 1, order= 1
class 9: size= 1, order= 2
Number of nonzeros in Ug†Ug - I: 0
All tests passed for this Ug.
Number of nonzeros in Ug†Ug - I: 0
All tests passed for this Ug.
Number of nonzeros in Ug†Ug - I: 0
All tests passed for this Ug.
Number of nonzeros in Ug†Ug - I: 0
All tests passed for this Ug.
Number of nonzeros in Ug†Ug - I: 0
All tests passed for this Ug.
Number of nonzeros in Ug

In [175]:
eigenvalues, eigenvectors = eigh(H.toarray())

In [181]:
# Check irrep membership for 5 consecutive eigenvectors (potential scars)
#scar_indices = [1266, 1267, 1268, 1269, 1270]
scar_indices = [307, 577, 984, 1012, 1266, 1267, 1268, 1269, 1270, 1297, 1490, 1527, 2057, 2104, 2585, 2643, 2921, 3159, 3520]

for idx in scar_indices:
    v = eigenvectors[:, idx]
    E = eigenvalues[idx]
    print(f"\nEigenvector index: {idx}, Eigenvalue: {E:.6f}")
    
    # Check symmetry properties
    mag_avg = check_magnetization_sector(v, N)
    parity_sector = check_parity_sector(v, N)
    
    print(f"  Magnetization: {mag_avg:.6f}")
    print(f"  Parity: {parity_sector}")


Eigenvector index: 307, Eigenvalue: -16.874640
  Magnetization: -3.683869
  Parity: even

Eigenvector index: 577, Eigenvalue: -12.874640
  Magnetization: -3.683869
  Parity: even

Eigenvector index: 984, Eigenvalue: -8.644318
  Magnetization: -1.929929
  Parity: odd

Eigenvector index: 1012, Eigenvalue: -8.271258
  Magnetization: -0.133804
  Parity: even

Eigenvector index: 1266, Eigenvalue: -6.000000
  Magnetization: -0.000000
  Parity: even

Eigenvector index: 1267, Eigenvalue: -6.000000
  Magnetization: -0.000000
  Parity: even

Eigenvector index: 1268, Eigenvalue: -6.000000
  Magnetization: -0.000000
  Parity: even

Eigenvector index: 1269, Eigenvalue: -6.000000
  Magnetization: -0.000000
  Parity: even

Eigenvector index: 1270, Eigenvalue: -6.000000
  Magnetization: -0.000000
  Parity: even

Eigenvector index: 1297, Eigenvalue: -5.854318
  Magnetization: -1.904904
  Parity: odd

Eigenvector index: 1490, Eigenvalue: -4.271258
  Magnetization: -0.133804
  Parity: even

Eigenvector 

In [177]:
target_E0 = -4.0
# Generate initial block in Sz=0, even parity, spatially symmetric sector
vc0 = random_block_in_sz0_even_parity(N, block_size=5)

Phi, evals = chebyshev_filter_block_numpy(H, vc0, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=10000, pad=0.0, use_jackson=False)
for i in range(len(evals)):
    print(f"Eigenvalue {i}: {evals[i]:.6f}")
    print(f"Support size (non-zero elements): {np.count_nonzero(np.abs(Phi[:, i]) > 1e-10)}")
'''
# Orthonormality of columns in Phi
G = Phi.conj().T @ Phi
I = np.eye(Phi.shape[1], dtype=np.complex128)
diag_err = np.max(np.abs(np.diag(G) - 1.0))
offdiag_mask = ~np.eye(Phi.shape[1], dtype=bool)
offdiag_max = np.max(np.abs(G - I)[offdiag_mask]) if Phi.shape[1] > 1 else 0.0
print(f"Phi orthonormality: max |diag-1| = {diag_err:.3e}, max |offdiag| = {offdiag_max:.3e}")
'''

Phi, evals = chebyshev_filter_block_numpy(H, Phi, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=10000, pad=0.0, use_jackson=False)
for i in range(len(evals)):
    print(f"Eigenvalue {i}: {evals[i]:.6f}")
    print(f"Support size (non-zero elements): {np.count_nonzero(np.abs(Phi[:, i]) > 1e-10)}")

Eigenvalue 0: -4.004616
Support size (non-zero elements): 2048
Eigenvalue 1: -4.003689
Support size (non-zero elements): 2048
Eigenvalue 2: -4.001396
Support size (non-zero elements): 2048
Eigenvalue 3: -3.998894
Support size (non-zero elements): 2048
Eigenvalue 4: -3.973585
Support size (non-zero elements): 2048
Eigenvalue 0: -4.004754
Support size (non-zero elements): 2048
Eigenvalue 1: -4.004721
Support size (non-zero elements): 2048
Eigenvalue 2: -4.004515
Support size (non-zero elements): 2048
Eigenvalue 3: -3.999986
Support size (non-zero elements): 2048
Eigenvalue 4: -3.980174
Support size (non-zero elements): 2048


In [186]:
v0 = np.random.randn(H.shape[0])
target_E0 = -4.0
vc, Ec = chebyshev_filter_v0_numpy(H, v0, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=100000, pad=0.05, use_jackson=True)
print(Ec, np.count_nonzero(np.abs(vc) > 1e-10))
vc1, Ec1 = chebyshev_filter_v0_numpy(H, vc, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=100000, pad=0.05, use_jackson=True)
print(Ec1, np.count_nonzero(np.abs(vc1) > 1e-10))
vc2, Ec2 = chebyshev_filter_v0_numpy(H, vc1, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=100000, pad=0.05, use_jackson=True)
print(Ec2, np.count_nonzero(np.abs(vc2) > 1e-10))
vc3, Ec3 = chebyshev_filter_v0_numpy(H, vc2, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=100000, pad=0.05, use_jackson=True)
print(Ec3, np.count_nonzero(np.abs(vc3) > 1e-10))
vc4, Ec4 = chebyshev_filter_v0_numpy(H, vc3, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=100000, pad=0.05, use_jackson=True)
print(Ec4, np.count_nonzero(np.abs(vc4) > 1e-10))
vc5, Ec5 = chebyshev_filter_v0_numpy(H, vc4, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=100000, pad=0.05, use_jackson=True)
print(Ec5, np.count_nonzero(np.abs(vc5) > 1e-10))

-3.9999970766835244 4096
-4.000000000002083 3578
-4.0 1792
-4.000000000000001 720
-3.999999999999999 720
-4.0 720


In [187]:
Et = eigenvalues[1527]
vt = eigenvectors[:,1527]
print(Et, np.count_nonzero(np.abs(vt) > 1e-10))

-4.000000000000002 720


In [188]:
print((np.abs(np.dot(vt.conj(), vc4)))**2)

1.0
